In [1]:
from pprint import pprint

def test_oracle(codes, crels, orcl_fact, verbose=False):
    
    crels = set(crels)
    if verbose:
        prn_fun = lambda s="": print(s)
    else:
        prn_fun = lambda s="": None
    
    stack = Stack(False)
    stack.push(ROOT)
    parser = Parser(stack)
    oracle = orcl_fact(crels, parser)

    prn_fun("DEPS")
    for crel in sorted(crels):
        prn_fun("\t" + str(crel))
    prn_fun()

    PAD = 20
    LINE = PAD + len(ROOT) + 2 * len(codes) + 1

    for buffer in codes:
        prn_fun("-" * LINE)
        prn_fun(buffer)
        prn_fun("-" * LINE)

        while True:
            tos = stack.tos()
            if not oracle.consult(tos, buffer):
                prn_fun(parser.actions[-1].ljust(PAD) + " || STACK : " + str(stack))
                break

            prn_fun(parser.actions[-1].ljust(PAD) + " || STACK : " + str(stack))
            if stack.len() == 0:
                prn_fun("Empty stack, stopping")
                break

    prn_fun()
    prn_fun("*" * LINE)
    prn_fun("Stack")
    prn_fun("\t" + str(stack))
    deps = parser.get_dependencies()
    prn_fun("DEPS Actual")
    for crel in sorted(crels):
        prn_fun("\t" + str(crel))
    prn_fun("DEPS Pred")
    for dep in sorted(deps):
        prn_fun("\t" + str(dep))
    prn_fun("Actions")
    for a in parser.actions:
        prn_fun("\t" + a)
    prn_fun()
    prn_fun("Ordered Match?    " + str(set(deps) == crels))

    ndeps = norm_arcs(deps)
    ncrels = norm_arcs(crels)
    diff = (ndeps - ncrels).union(ncrels - ndeps)
    success = (len(diff) == 0)
    prn_fun("Un Ordered Match? " + str(success))
    if diff:
        prn_fun(diff)
    return success

In [2]:
class Stack(object):
    def __init__(self, verbose=False):    
        self.stack = []
        self.verbose = verbose
    
    def tos(self):
        if self.len() == 0:
            return None
        #assert self.len() > 0, "Can't peek when stack is empty"
        return self.stack[-1]
    
    def pop(self):
        assert self.len() > 0, "Can't pop when stack is empty"
        item = self.stack.pop()
        if self.verbose:
            print("POPPING: %s" % item)
            print("LEN:     %i" % len(self.stack))
        return item
    
    def push(self, item):
        self.stack.append(item)
        if self.verbose:
            print("PUSHING: %s" % item)
            print("LEN:     %i" % len(self.stack))
    
    def len(self):
        return len(self.stack)

    def contains(self, item):
        return item in self.stack
    
    def __repr__(self):
        return "|".join(self.stack)

In [9]:
ROOT = "root"

def norm_arc(arc):
    return tuple(sorted(arc))

def norm_arcs(arcs):
    return set(map(norm_arc, arcs))

class Parser(object):
    def __init__(self, stack):
        self.stack = stack
        self.arcs = []
        self.normed_arcs = set()
        # nodes with heads
        self.children = set()
        self.actions = []
        
    def get_dependencies(self):
        return [(l,r) for (l,r) in self.arcs if r != ROOT and l != ROOT]
        
    def left_arc(self, buffer):
        tos = self.stack.pop()
        #Pre-condition
        #assert self.has_head(tos) == False
        arc = (tos,buffer)
        n_arc = norm_arc(arc)
        assert n_arc not in self.normed_arcs, "Arc already processed %s" % (n_arc)
        self.arcs.append(arc)
        self.normed_arcs.add(arc)
        self.children.add(tos)
        self.actions.append("L ARC   : " + tos + "->" + buffer)
        
    def right_arc(self, buffer):
        tos = self.stack.tos()
        #normalize arc
        arc = (buffer,tos)
        n_arc = norm_arc(arc)
        assert n_arc not in self.normed_arcs, "Arc already processed %s" % (n_arc)
        self.arcs.append(arc)
        self.normed_arcs.add(n_arc)
        self.actions.append("R ARC   : " + tos + "<-" + buffer)
        self.children.add(buffer)
        self.stack.push(buffer)
        
    def reduce(self):
        tos = self.stack.pop()
        #assert self.has_head(tos) == True
        self.actions.append("REDUCE  : Pop  %s" % tos)
        
    def shift(self, buffer):
        self.stack.push(buffer)
        self.actions.append("SHIFT   : Push %s" % buffer)
    
    def skip(self, buffer):
        self.actions.append("SKIP    : item %s" % buffer)
    
    def has_head(self, item):
        return item in self.children
    
    def in_stack(self, item):
        return self.stack.contains(item)

In [10]:
from collections import defaultdict

SHIFT = "Shift"
REDUCE = "Reduce"
LARC = "LArc"
RARC = "Rarc"
SKIP = "Skip"

class Oracle(object):
    
    def __init__(self, crels, parser):
        self.parser = parser
        self.crels = norm_arcs(crels)
        self.mapping = self.build_mappings(crels)
    
    def build_mappings(self, pairs):
        mapping = defaultdict(set)
        for c,res in pairs:
            mapping[c].add(res)
            mapping[res].add(c)
        return mapping

    def cont(self, action):
        # continue parsing if REDUCE or LARC
        return action in (REDUCE,LARC)
    
    def remove_relation(self, a,b):
        self.mapping[a].remove(b)
        if len(self.mapping[a]) == 0:
            del self.mapping[a]
        self.mapping[b].remove(a)
        if len(self.mapping[b]) == 0:
            del self.mapping[b]
    
    def consult(self, tos, buffer):
        """
        Performs optimal decision for parser
        If true, continue processing, else Consume Buffer
        """
        parser = self.parser
        a,b = norm_arc((tos, buffer))
        if (a,b) in self.crels:
            # TOS has arcs remaining? If so, we need RARC, else LARC
            if len(self.mapping[tos]) == 1:
                parser.left_arc(buffer)
                self.remove_relation(tos, buffer)
                return self.cont(LARC)
            else:
                parser.right_arc(buffer)
                self.remove_relation(tos, buffer)
                return self.cont(RARC)
        else:
            if buffer not in self.mapping:
                parser.skip(buffer)
                return self.cont(SKIP)
            # If the buffer has relations further down in the stack, we need to POP the TOS
            for item in self.mapping[buffer]:
                if item == tos:
                    continue
                if parser.in_stack(item):
                    parser.reduce()
                    return self.cont(REDUCE)
            #end for
            #ELSE
            parser.shift(buffer)
            return self.cont(SHIFT)

In [11]:
test_pairs = []

test_pairs.append([
    ("A","B"),
])
test_pairs.append([
    ("A","B"),
    ("B","C"),
])
#C->B->A
test_pairs.append([
    ("C","B"),
    ("B","A"),
])
test_pairs.append([
    ("A","C"),
    ("B","C"),
])
test_pairs.append([
    ("A","B"),
    ("C","B"),
])
test_pairs.append([
    ("B","A"),
    ("B","C"),
])
test_pairs.append([
    ("A","C"),
    ("C","B"),
])

# Hard - has to flip relation
test_pairs.append([
    ("A","D"),
    ("D","B"),
    ("B","C"),
])
test_pairs.append([
    ("D","A"),
    ("D","B"),
    ("B","C"),
])
test_pairs.append([
    ("D","A"),
    ("B","D"),
    ("B","C"),
])

test_pairs.append([
    ("A","E"),
    ("E","B"),
    ("B","D"),
    ("D","C"),
])
test_pairs.append([
    ("A","D"),
    ("D","B"),
    ("B","C"),
    ("A", "F"),
    ("A", "E"),
])

test_pairs.append([
    ("A","D"),
    ("D","B"),
    ("B","C"),
    ("A", "F"),
    ("E", "F"),
])

oracle_fact = Oracle
for pairs in test_pairs:
    try:
        success = test_oracle("ABCDEF", pairs, oracle_fact, verbose=False)
    except:
        success = False
        
    if not success:
        print("Error for relations:")
        pprint(pairs)
        print()
        success = test_oracle("ABCDEF", pairs, oracle_fact, verbose=True)

## Visualize Parse for Tricker Graphs

### <span style="color:red">Doesn't Handle Cycles</span>
- So we remove the condition about only having a single parent

In [12]:
#[('1', '3'), ('1', '50'), ('3', '50')]
#['50', '1', '3']
pairs =[
    ("B","A"),
    ("B","C"),
    ("C","A"),
]
test_oracle("ABCDEF", pairs, Oracle, verbose=True)

DEPS
	('B', 'A')
	('B', 'C')
	('C', 'A')

-------------------------------------
A
-------------------------------------
SHIFT   : Push A     || STACK : root|A
-------------------------------------
B
-------------------------------------
R ARC   : A<-B       || STACK : root|A|B
-------------------------------------
C
-------------------------------------
L ARC   : B->C       || STACK : root|A
L ARC   : A->C       || STACK : root
SKIP    : item C     || STACK : root
-------------------------------------
D
-------------------------------------
SKIP    : item D     || STACK : root
-------------------------------------
E
-------------------------------------
SKIP    : item E     || STACK : root
-------------------------------------
F
-------------------------------------
SKIP    : item F     || STACK : root

*************************************
Stack
	root
DEPS Actual
	('B', 'A')
	('B', 'C')
	('C', 'A')
DEPS Pred
	('A', 'C')
	('B', 'A')
	('B', 'C')
Actions
	SHIFT   : Push A
	R ARC   : A<-B

True

In [13]:
pairs =[
    ("A","D"),
    ("D","B"),
    ("B","C"),
]
test_oracle("ABCDEF", pairs, Oracle, verbose=True)

DEPS
	('A', 'D')
	('B', 'C')
	('D', 'B')

-------------------------------------
A
-------------------------------------
SHIFT   : Push A     || STACK : root|A
-------------------------------------
B
-------------------------------------
SHIFT   : Push B     || STACK : root|A|B
-------------------------------------
C
-------------------------------------
R ARC   : B<-C       || STACK : root|A|B|C
-------------------------------------
D
-------------------------------------
REDUCE  : Pop  C     || STACK : root|A|B
L ARC   : B->D       || STACK : root|A
L ARC   : A->D       || STACK : root
SKIP    : item D     || STACK : root
-------------------------------------
E
-------------------------------------
SKIP    : item E     || STACK : root
-------------------------------------
F
-------------------------------------
SKIP    : item F     || STACK : root

*************************************
Stack
	root
DEPS Actual
	('A', 'D')
	('B', 'C')
	('D', 'B')
DEPS Pred
	('A', 'D')
	('B', 'D')
	('C',

True

## Non Projective Parse Should Fail Test

In [14]:
pairs =[
    ("A","C"),
    ("B","E"),
]
try:
    success = test_oracle("ABCDEF", pairs, Oracle, verbose=True)
except Exception as e:
    success = False
    raise e
assert success == False

DEPS
	('A', 'C')
	('B', 'E')

-------------------------------------
A
-------------------------------------
SHIFT   : Push A     || STACK : root|A
-------------------------------------
B
-------------------------------------
SHIFT   : Push B     || STACK : root|A|B
-------------------------------------
C
-------------------------------------
REDUCE  : Pop  B     || STACK : root|A
L ARC   : A->C       || STACK : root
SKIP    : item C     || STACK : root
-------------------------------------
D
-------------------------------------
SKIP    : item D     || STACK : root
-------------------------------------
E
-------------------------------------
SHIFT   : Push E     || STACK : root|E
-------------------------------------
F
-------------------------------------
SKIP    : item F     || STACK : root|E

*************************************
Stack
	root|E
DEPS Actual
	('A', 'C')
	('B', 'E')
DEPS Pred
	('A', 'C')
Actions
	SHIFT   : Push A
	SHIFT   : Push B
	REDUCE  : Pop  B
	L ARC   : A->C
	SKIP

## Test on Real Causal Relations (Limit to 2 or More Relations in a Sentence)

In [17]:
def normalize_cr(cr):
    return tuple(cr.replace("Causer:","").replace("Result:","").split("->"))

In [18]:
normalize_cr('Causer:14->Result:50')

('14', '50')

## 4 Errors Below Look Are from Non-Projective Parses

In [19]:
errors = 0
exs = []
for supported_causal, tag_seq in relations[:]:
    supported_causal = sorted(supported_causal)
    crels = [normalize_cr(crel) for crel in supported_causal]

    try:
        success = test_oracle(tag_seq, crels, Oracle, verbose=False)
    except Exception as e:
        exs.append(e)
        success = False
        
    if not success:
        errors += 1
        print("Error for relations:")
        pprint(crels)
        pprint(tag_seq)
        #print()
        #success = test_oracle(tag_seq, crels, Oracle, verbose=True)
        #break

Error for relations:
[('1', '50'), ('1', '7'), ('3', '50'), ('3', '7')]
['1', '3', '50', '7']
Error for relations:
[('1', '4'), ('1', '50'), ('3', '4'), ('3', '50')]
['1', '3', '4', '50']
Error for relations:
[('13', '6'), ('13', '7'), ('3', '6'), ('3', '7'), ('7', '50')]
['6', '3', '13', '7', '50']
Error for relations:
[('12', '11'), ('13', '11'), ('13', '14')]
['12', '13', '11', '14']


**NOTES**
With only 4 errors as 4 missed relations, hardly worth worrying about. 
One solution would be to train a forward and a backward parser, parse the sentence in both directions and merge the deps. In each case that would pick up all deps.

### How Many Errors?

In [16]:
errors, len(relations)

NameError: name 'errors' is not defined

## <span style="color:red">NEED to determine if all errors are non-projective<span>

In [15]:
relations = [
(['Causer:1->Result:50', 'Causer:11->Result:50', 'Causer:13->Result:50'],['50', '13', '1', '11']),
(['Causer:1->Result:50', 'Causer:3->Result:1'],['50', '1', '3']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['3', '50', '13']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:50'],['11', '12', '13', '50']),
(['Causer:5b->Result:50', 'Causer:7->Result:5b'],['7', '5b', '50']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:13', 'Causer:11->Result:3', 'Causer:11->Result:50'],['11', '3', '13', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:50', 'Causer:1->Result:7', 'Causer:3->Result:50', 'Causer:3->Result:7'],['1', '3', '50', '7']),
(['Causer:7->Result:14', 'Causer:7->Result:50'],['50', '7', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:13', 'Causer:13->Result:50'],['50', '13', '11']),
(['Causer:11->Result:13', 'Causer:11->Result:14'],['13', '11', '14']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:3->Result:5', 'Causer:5->Result:50', 'Causer:5b->Result:50'],['3', '5', '50', '5b']),
(['Causer:1->Result:3', 'Causer:3->Result:4'],['1', '3', '4']),
(['Causer:3->Result:14', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:3', 'Causer:1->Result:50'],['1', '3', '50']),
(['Causer:5->Result:7', 'Causer:7->Result:50'],['50', '7', '5']),
(['Causer:1->Result:7', 'Causer:3->Result:7'],['7', '1', '3']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:2->Result:3'],['3', '1', '2']),
(['Causer:11->Result:14', 'Causer:14->Result:50'],['50', '11', '14']),
(['Causer:3->Result:50', 'Causer:6->Result:50'],['50', '3', '6']),
(['Causer:6->Result:50', 'Causer:6->Result:7'],['6', '50', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:12->Result:13', 'Causer:13->Result:14'],['13', '11', '12', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:12->Result:13'],['13', '11', '12']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:11->Result:50', 'Causer:13->Result:50'],['50', '11', '13']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['3', '50', '4']),
(['Causer:11->Result:14', 'Causer:13->Result:14'],['14', '11', '13']),
(['Causer:1->Result:7', 'Causer:7->Result:50'],['1', '7', '50']),
(['Causer:1->Result:50', 'Causer:13->Result:50', 'Causer:7->Result:50'],['50', '1', '13', '7']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50', 'Causer:5->Result:50'],['1', '3', '5', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:4', 'Causer:3->Result:4'],['4', '3', '11']),
(['Causer:11->Result:50', 'Causer:13->Result:50'],['50', '11', '13']),
(['Causer:11->Result:3', 'Causer:11->Result:50'],['50', '11', '3']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:1->Result:50'],['1', '50', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:14'],['11', '12', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:13', 'Causer:12->Result:13'],['11', '13', '12']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:14'],['11', '12', '13', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:4', 'Causer:4->Result:14'],['1', '3', '4', '14']),
(['Causer:3->Result:50', 'Causer:4->Result:11'],['3', '50', '4', '11']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:1->Result:50', 'Causer:7->Result:50'],['7', '50', '1']),
(['Causer:1->Result:4', 'Causer:3->Result:4'],['1', '4', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:11->Result:50', 'Causer:3->Result:50'],['50', '1', '3', '11']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:4', 'Causer:4->Result:50'],['50', '4', '1']),
(['Causer:6->Result:14', 'Causer:7->Result:50'],['6', '14', '50', '7']),
(['Causer:3->Result:5', 'Causer:3->Result:50'],['50', '3', '5']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['50', '3', '13']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:7->Result:50'],['50', '1', '7']),
(['Causer:11->Result:14', 'Causer:13->Result:14', 'Causer:3->Result:14', 'Causer:4->Result:14'],['14', '3', '4', '11', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['13', '11', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:1->Result:3', 'Causer:3->Result:7', 'Causer:7->Result:50'],['1', '3', '7', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:1->Result:50', 'Causer:7->Result:50'],['50', '1', '3', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:7->Result:50'],['50', '1', '7']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:13->Result:50', 'Causer:3->Result:50', 'Causer:7->Result:50'],['50', '3', '13', '7']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['13', '3', '50']),
(['Causer:11->Result:12', 'Causer:11->Result:6', 'Causer:12->Result:50'],['50', '6', '11', '12']),
(['Causer:6->Result:50', 'Causer:7->Result:50'],['6', '50', '7']),
(['Causer:14->Result:6', 'Causer:6->Result:50'],['14', '6', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['6', '14', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:14'],['3', '1', '14']),
(['Causer:14->Result:50', 'Causer:14->Result:7'],['50', '14', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:7'],['1', '3', '7']),
(['Causer:14->Result:50', 'Causer:7->Result:14'],['7', '14', '50']),
(['Causer:3->Result:50', 'Causer:6->Result:50'],['50', '3', '6']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:13', 'Causer:5->Result:5b'],['3', '4', '13', '5', '5b']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:3->Result:5'],['1', '3', '5']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:13->Result:6', 'Causer:3->Result:6'],['6', '3', '13']),
(['Causer:3->Result:4', 'Causer:4->Result:5b'],['3', '4', '5b']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['50', '3', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:50', 'Causer:3->Result:6'],['50', '3', '6']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:5->Result:7', 'Causer:7->Result:50'],['5', '7', '50']),
(['Causer:3->Result:50', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:13->Result:50'],['50', '1', '13']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['50', '6', '7']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:50'],['12', '11', '13', '50']),
(['Causer:13->Result:50', 'Causer:3->Result:50', 'Causer:7->Result:50'],['50', '7', '13', '3']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:11->Result:50', 'Causer:13->Result:50'],['11', '13', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:12', 'Causer:11->Result:13'],['11', '13', '12']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:14'],['11', '12', '13', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:14->Result:50', 'Causer:6->Result:14', 'Causer:7->Result:50'],['50', '6', '14', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:1'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:7->Result:50'],['50', '1', '5', '7']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:5->Result:5b', 'Causer:7->Result:50'],['5b', '5', '50', '7']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:1->Result:2', 'Causer:2->Result:3'],['1', '2', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:6->Result:50', 'Causer:6->Result:7'],['50', '6', '7']),
(['Causer:1->Result:2', 'Causer:1->Result:3'],['1', '2', '3']),
(['Causer:1->Result:11', 'Causer:11->Result:13', 'Causer:13->Result:14', 'Causer:14->Result:7'],['1', '11', '13', '14', '7']),
(['Causer:3->Result:4', 'Causer:3->Result:50', 'Causer:4->Result:5'],['3', '50', '4', '5']),
(['Causer:3->Result:4', 'Causer:3->Result:5b'],['3', '4', '5b']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:2', 'Causer:2->Result:3'],['1', '2', '3']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:12->Result:13'],['11', '13', '12']),
(['Causer:1->Result:3', 'Causer:1->Result:50', 'Causer:11->Result:13', 'Causer:13->Result:50'],['50', '1', '3', '11', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:13->Result:50', 'Causer:5->Result:50'],['50', '13', '5']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:4', 'Causer:4->Result:5b', 'Causer:5b->Result:50'],['4', '3', '5b', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:14'],['11', '12', '13', '14']),
(['Causer:6->Result:50', 'Causer:6->Result:7'],['50', '6', '7']),
(['Causer:5->Result:50', 'Causer:6->Result:50'],['6', '50', '5']),
(['Causer:11->Result:12', 'Causer:13->Result:12'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:13->Result:50', 'Causer:3->Result:50', 'Causer:4->Result:50'],['50', '13', '3', '4']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:50', 'Causer:7->Result:50'],['50', '7', '1']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:13->Result:50', 'Causer:4->Result:50'],['50', '1', '4', '13']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['4', '50', '3']),
(['Causer:11->Result:50', 'Causer:13->Result:50', 'Causer:4->Result:50'],['11', '13', '4', '50']),
(['Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:7->Result:50'],['50', '1', '5', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:13->Result:14', 'Causer:13->Result:50', 'Causer:14->Result:50'],['13', '50', '14']),
(['Causer:11->Result:50', 'Causer:3->Result:50'],['50', '3', '11']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:1->Result:13', 'Causer:13->Result:50'],['1', '13', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:3->Result:50', 'Causer:50->Result:7'],['50', '3', '7']),
(['Causer:50->Result:7', 'Causer:7->Result:50'],['50', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:50', 'Causer:50->Result:1'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:11->Result:50', 'Causer:13->Result:50', 'Causer:3->Result:50', 'Causer:4->Result:50', 'Causer:7->Result:50'],['50', '11', '3', '4', '13', '7', '1']),
(['Causer:12->Result:13', 'Causer:13->Result:14'],['12', '13', '14']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:1->Result:3', 'Causer:1->Result:50', 'Causer:1->Result:7'],['50', '1', '3', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:7'],['1', '3', '7']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:5b', 'Causer:3->Result:6'],['6', '3', '11', '12', '5b']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:7->Result:50'],['7', '5', '1', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:11->Result:13'],['1', '3', '11', '13']),
(['Causer:13->Result:50', 'Causer:7->Result:50'],['50', '13', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:12->Result:13'],['13', '11', '12']),
(['Causer:1->Result:50', 'Causer:2->Result:3', 'Causer:3->Result:50'],['1', '50', '2', '3']),
(['Causer:3->Result:5', 'Causer:5->Result:5b'],['3', '5', '5b']),
(['Causer:11->Result:13', 'Causer:11->Result:3'],['11', '3', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:14'],['11', '12', '13', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:7->Result:50'],['50', '1', '7']),
(['Causer:1->Result:2', 'Causer:2->Result:3'],['1', '2', '3']),
(['Causer:3->Result:50', 'Causer:7->Result:50'],['3', '50', '7']),
(['Causer:1->Result:6', 'Causer:6->Result:7'],['1', '6', '7']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:12->Result:13'],['11', '13', '12']),
(['Causer:12->Result:7', 'Causer:7->Result:50'],['50', '7', '12']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:1->Result:2', 'Causer:4->Result:14'],['1', '2', '4', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:11->Result:50'],['1', '11', '50']),
(['Causer:11->Result:13', 'Causer:12->Result:13'],['11', '13', '12']),
(['Causer:1->Result:50', 'Causer:11->Result:50'],['50', '1', '11']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:1->Result:2', 'Causer:2->Result:50'],['1', '2', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:3->Result:50', 'Causer:7->Result:50'],['50', '3', '7']),
(['Causer:5->Result:5b', 'Causer:5b->Result:14'],['5', '5b', '14']),
(['Causer:5->Result:7', 'Causer:5b->Result:7'],['7', '5', '5b']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['50', '7', '3']),
(['Causer:1->Result:3', 'Causer:3->Result:7'],['1', '3', '7']),
(['Causer:4->Result:5b', 'Causer:5b->Result:7', 'Causer:7->Result:50'],['4', '5b', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:4'],['1', '3', '4']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:13->Result:4'],['11', '12', '13', '4']),
(['Causer:1->Result:2', 'Causer:1->Result:3'],['2', '1', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50', 'Causer:7->Result:5b'],['6', '7', '50', '5b']),
(['Causer:3->Result:4', 'Causer:4->Result:7'],['3', '4', '7']),
(['Causer:11->Result:50', 'Causer:13->Result:50'],['11', '50', '13']),
(['Causer:11->Result:12', 'Causer:11->Result:13', 'Causer:12->Result:13'],['13', '11', '12']),
(['Causer:11->Result:50', 'Causer:3->Result:50', 'Causer:4->Result:50'],['50', '3', '4', '11']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['3', '50', '4']),
(['Causer:4->Result:50', 'Causer:5->Result:50'],['50', '5', '4']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['13', '11', '12']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['13', '50', '3']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '14', '6']),
(['Causer:1->Result:50', 'Causer:13->Result:50', 'Causer:7->Result:50'],['1', '13', '7', '50']),
(['Causer:2->Result:3', 'Causer:3->Result:4', 'Causer:4->Result:5'],['2', '3', '4', '5']),
(['Causer:3->Result:14', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:4->Result:50', 'Causer:7->Result:50'],['4', '7', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:7', 'Causer:7->Result:50'],['3', '4', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:50', 'Causer:7->Result:50'],['50', '7', '11']),
(['Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:7->Result:50'],['50', '1', '5', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:4', 'Causer:4->Result:14'],['1', '3', '4', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:12->Result:50', 'Causer:12->Result:7', 'Causer:4->Result:7'],['50', '12', '7', '4']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['50', '13', '3']),
(['Causer:5->Result:50', 'Causer:7->Result:50'],['5', '50', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:4', 'Causer:1->Result:50', 'Causer:3->Result:4', 'Causer:3->Result:50'],['1', '3', '4', '50']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:7'],['7', '3', '1']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:13->Result:14', 'Causer:4->Result:5', 'Causer:4->Result:50'],['13', '14', '4', '5', '50']),
(['Causer:4->Result:50', 'Causer:5b->Result:5'],['4', '50', '5b', '5']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:5->Result:50', 'Causer:6->Result:7'],['6', '7', '5', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:7', 'Causer:7->Result:50'],['3', '1', '7', '50']),
(['Causer:11->Result:13', 'Causer:13->Result:5', 'Causer:5->Result:7', 'Causer:7->Result:50'],['11', '13', '5', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:4->Result:5', 'Causer:4->Result:5b'],['5', '4', '5b']),
(['Causer:1->Result:3', 'Causer:3->Result:6', 'Causer:6->Result:50'],['1', '3', '6', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:13->Result:50', 'Causer:3->Result:50'],['50', '1', '3', '13']),
(['Causer:12->Result:13', 'Causer:13->Result:50'],['12', '13', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:5', 'Causer:3->Result:50'],['3', '5', '50']),
(['Causer:1->Result:50', 'Causer:11->Result:50'],['50', '1', '11']),
(['Causer:11->Result:13', 'Causer:13->Result:50'],['50', '13', '11']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:13->Result:50', 'Causer:3->Result:13'],['3', '13', '50']),
(['Causer:11->Result:12', 'Causer:11->Result:50', 'Causer:12->Result:13'],['11', '50', '12', '13']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:13->Result:50', 'Causer:3->Result:50'],['50', '3', '13']),
(['Causer:5->Result:5b', 'Causer:5b->Result:50'],['5', '5b', '50']),
(['Causer:4->Result:5b', 'Causer:5b->Result:50'],['4', '5b', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:13', 'Causer:12->Result:13'],['11', '13', '12']),
(['Causer:2->Result:3', 'Causer:3->Result:50'],['2', '3', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:7->Result:50'],['50', '1', '5', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:5b', 'Causer:5->Result:5b', 'Causer:5b->Result:50'],['1', '3', '5b', '5', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:13', 'Causer:13->Result:50'],['50', '13', '11']),
(['Causer:6->Result:50', 'Causer:6->Result:7'],['50', '6', '7']),
(['Causer:12->Result:13', 'Causer:13->Result:14'],['12', '13', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:6', 'Causer:6->Result:7'],['3', '6', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:3->Result:50', 'Causer:4->Result:50'],['50', '3', '4']),
(['Causer:11->Result:13', 'Causer:13->Result:50'],['13', '11', '50']),
(['Causer:11->Result:50', 'Causer:13->Result:50'],['11', '50', '13']),
(['Causer:3->Result:5', 'Causer:5->Result:50'],['3', '5', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:2->Result:3', 'Causer:3->Result:5'],['3', '5', '2']),
(['Causer:1->Result:3', 'Causer:3->Result:7'],['3', '1', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:11->Result:13', 'Causer:11->Result:3', 'Causer:13->Result:50'],['11', '3', '13', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '50', '1']),
(['Causer:1->Result:2', 'Causer:1->Result:3', 'Causer:1->Result:50'],['50', '1', '3', '2']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:7->Result:50'],['50', '1', '7']),
(['Causer:3->Result:4', 'Causer:4->Result:7'],['3', '4', '7']),
(['Causer:5->Result:50', 'Causer:7->Result:50'],['50', '7', '5']),
(['Causer:1->Result:3', 'Causer:3->Result:5'],['1', '3', '5']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:5b->Result:7', 'Causer:7->Result:50'],['7', '5b', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:12->Result:13', 'Causer:13->Result:14'],['12', '13', '14']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:14'],['11', '12', '13', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:7', 'Causer:5b->Result:50'],['3', '7', '5b', '50']),
(['Causer:1->Result:50', 'Causer:5->Result:50'],['50', '1', '5']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:13', 'Causer:13->Result:50'],['50', '11', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:2', 'Causer:1->Result:50', 'Causer:2->Result:3'],['50', '1', '2', '3']),
(['Causer:1->Result:2', 'Causer:2->Result:3', 'Causer:3->Result:50'],['1', '2', '3', '50']),
(['Causer:1->Result:3', 'Causer:5->Result:3', 'Causer:5->Result:50'],['50', '5', '3', '1']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:7->Result:50'],['1', '5', '7', '50']),
(['Causer:11->Result:50', 'Causer:13->Result:50'],['11', '13', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['50', '6', '7']),
(['Causer:1->Result:2', 'Causer:1->Result:3'],['1', '2', '3']),
(['Causer:4->Result:50', 'Causer:5b->Result:50'],['4', '50', '5b']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:5', 'Causer:3->Result:5'],['1', '5', '3']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:4->Result:7', 'Causer:7->Result:50'],['4', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:5'],['3', '4', '5']),
(['Causer:1->Result:50', 'Causer:11->Result:50'],['50', '1', '11']),
(['Causer:3->Result:7', 'Causer:4->Result:14', 'Causer:7->Result:4'],['3', '7', '4', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:5'],['1', '3', '5']),
(['Causer:1->Result:50', 'Causer:5->Result:50'],['50', '1', '5']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['50', '3', '4']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:3->Result:7', 'Causer:4->Result:7'],['3', '7', '4']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '50', '1']),
(['Causer:11->Result:12', 'Causer:12->Result:50'],['11', '12', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:13->Result:6', 'Causer:13->Result:7', 'Causer:3->Result:6', 'Causer:3->Result:7', 'Causer:7->Result:50'],['6', '3', '13', '7', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:6', 'Causer:6->Result:7'],['3', '4', '6', '7']),
(['Causer:1->Result:50', 'Causer:4->Result:50'],['1', '4', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13', 'Causer:13->Result:50'],['11', '12', '13', '50']),
(['Causer:1->Result:2', 'Causer:2->Result:1'],['1', '2']),
(['Causer:1->Result:50', 'Causer:11->Result:50'],['50', '1', '11']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:6->Result:14', 'Causer:6->Result:50'],['50', '6', '14']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:11->Result:6', 'Causer:6->Result:50'],['11', '6', '50']),
(['Causer:6->Result:7', 'Causer:7->Result:50'],['6', '7', '50']),
(['Causer:3->Result:6', 'Causer:6->Result:50'],['3', '6', '50']),
(['Causer:13->Result:14', 'Causer:13->Result:50'],['50', '13', '14']),
(['Causer:1->Result:3', 'Causer:13->Result:50'],['3', '1', '13', '50']),
(['Causer:13->Result:50', 'Causer:5->Result:13', 'Causer:7->Result:50'],['5', '13', '50', '7']),
(['Causer:3->Result:14', 'Causer:3->Result:5'],['3', '5', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:4->Result:5', 'Causer:5->Result:7'],['4', '5', '7']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:3->Result:4', 'Causer:3->Result:5'],['3', '5', '4']),
(['Causer:1->Result:50', 'Causer:3->Result:4'],['1', '50', '3', '4']),
(['Causer:1->Result:50', 'Causer:5->Result:50'],['5', '1', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14', 'Causer:7->Result:50'],['50', '6', '14', '7']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:3', 'Causer:3->Result:4'],['1', '3', '4']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:13->Result:50', 'Causer:7->Result:50'],['50', '13', '7']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:4->Result:5', 'Causer:4->Result:5b'],['5', '4', '5b']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:5', 'Causer:5->Result:50', 'Causer:5b->Result:50'],['3', '5', '50', '5b']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:3->Result:50', 'Causer:4->Result:50'],['50', '3', '4']),
(['Causer:11->Result:13', 'Causer:13->Result:50', 'Causer:3->Result:13'],['11', '3', '13', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:2', 'Causer:2->Result:50', 'Causer:3->Result:50'],['50', '3', '1', '2']),
(['Causer:14->Result:50', 'Causer:4->Result:14'],['50', '4', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:3->Result:5', 'Causer:5->Result:7', 'Causer:7->Result:50'],['3', '5', '7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:5', 'Causer:3->Result:50'],['3', '5', '1', '50']),
(['Causer:3->Result:4', 'Causer:3->Result:5'],['3', '5', '4']),
(['Causer:5b->Result:50', 'Causer:7->Result:5b'],['7', '5b', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '50', '3']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:3->Result:50', 'Causer:4->Result:3'],['50', '4', '3']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:5->Result:50'],['1', '5', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:11->Result:50', 'Causer:3->Result:50'],['50', '3', '11']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:11->Result:13', 'Causer:13->Result:14'],['11', '13', '14']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:50->Result:7', 'Causer:7->Result:50'],['7', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:11->Result:3', 'Causer:3->Result:6', 'Causer:6->Result:7'],['11', '3', '6', '7']),
(['Causer:11->Result:50', 'Causer:13->Result:50', 'Causer:3->Result:50'],['3', '11', '13', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:5'],['3', '5', '1']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:50', 'Causer:6->Result:50'],['3', '6', '50']),
(['Causer:1->Result:5', 'Causer:5->Result:5b'],['1', '5', '5b']),
(['Causer:13->Result:14', 'Causer:14->Result:50'],['13', '14', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:5'],['1', '3', '5']),
(['Causer:1->Result:3', 'Causer:3->Result:4', 'Causer:4->Result:14'],['1', '3', '4', '14']),
(['Causer:5->Result:5b', 'Causer:5b->Result:50'],['5', '5b', '50']),
(['Causer:1->Result:3', 'Causer:1->Result:50'],['50', '1', '3']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:5', 'Causer:5->Result:50'],['3', '4', '5', '50']),
(['Causer:1->Result:50', 'Causer:11->Result:50', 'Causer:6->Result:50'],['6', '1', '11', '50']),
(['Causer:1->Result:50', 'Causer:13->Result:50'],['50', '1', '13']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:5->Result:50'],['50', '1', '5']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:13', 'Causer:13->Result:7'],['11', '13', '7']),
(['Causer:3->Result:4', 'Causer:4->Result:14'],['3', '4', '14']),
(['Causer:11->Result:13', 'Causer:12->Result:13'],['11', '13', '12']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
(['Causer:6->Result:14', 'Causer:6->Result:50'],['50', '6', '14']),
(['Causer:1->Result:50', 'Causer:4->Result:14'],['1', '50', '4', '14']),
(['Causer:1->Result:50', 'Causer:11->Result:50', 'Causer:6->Result:50'],['6', '1', '11', '50']),
(['Causer:1->Result:2', 'Causer:2->Result:50'],['1', '2', '50']),
(['Causer:1->Result:50', 'Causer:11->Result:50'],['50', '1', '11']),
(['Causer:13->Result:14', 'Causer:14->Result:50'],['13', '14', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50', 'Causer:5->Result:50'],['50', '3', '1', '5']),
(['Causer:1->Result:50', 'Causer:3->Result:50', 'Causer:5->Result:50'],['50', '1', '5', '3']),
(['Causer:1->Result:2', 'Causer:2->Result:3', 'Causer:6->Result:7', 'Causer:7->Result:50'],['1', '2', '3', '6', '7', '50']),
(['Causer:4->Result:50', 'Causer:6->Result:50'],['50', '4', '6']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:3->Result:7', 'Causer:7->Result:50'],['3', '7', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:11->Result:12', 'Causer:12->Result:13'],['11', '12', '13']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '3', '1']),
(['Causer:1->Result:50', 'Causer:13->Result:50', 'Causer:6->Result:50'],['50', '13', '6', '1']),
(['Causer:12->Result:11', 'Causer:13->Result:11', 'Causer:13->Result:14'],['12', '13', '11', '14']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:3', 'Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:3->Result:4', 'Causer:3->Result:50'],['3', '4', '50']),
(['Causer:3->Result:4', 'Causer:4->Result:50'],['3', '4', '50']),
(['Causer:1->Result:3', 'Causer:3->Result:6'],['1', '3', '6']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['50', '1', '3']),
(['Causer:1->Result:3', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['3', '1', '50']),
(['Causer:1->Result:50', 'Causer:3->Result:50'],['1', '3', '50']),
(['Causer:14->Result:50', 'Causer:6->Result:14'],['50', '6', '14']),
]